In [1]:
import numpy as np
from qutip import *
from matplotlib import pyplot as plt

In [2]:
# Constants
h = 6.602e-34
hbar = h/(2*np.pi)
kB = 1.38e-23
eps_0 = 8.85e-12
c_0 = 299792458
e = 1.602e-19
m_e = 9.109e-31
a_0 = 0.53e-10
mu_B = e*hbar/(2*m_e)
T = 5

def boltzmann(E, T):
    return np.exp(-h*E / (kB*T))

In [3]:
N_orbs = 4 
egx = basis(N_orbs, 0)
egy = basis(N_orbs, 1)
sgx = egx*egy.dag() + egy*egx.dag()
sgy = -1j*egx*egy.dag() + 1j*egy*egx.dag()
sgz = egx*egx.dag() - egy*egy.dag()

eux = basis(N_orbs, 2)
euy = basis(N_orbs, 3)
sux = eux*euy.dag() + euy*eux.dag()
suy = -1j*eux*euy.dag() + 1j*euy*eux.dag()
suz = eux*eux.dag() - euy*euy.dag()

N_spins = 2
su = basis(N_spins,0)
sd = basis(N_spins,1)
Sz = su*su.dag() - sd*sd.dag()
N = N_orbs*N_spins

In [4]:
'''System Dynamics:
Orbital part comprising Spin-Orbit, Jahn Teller, Zeeman and Strain couplin
'''
nu_Orb = 407e3
HOrb = Qobj(np.zeros((N_orbs, N_orbs)))
HOrb += nu_Orb*(eux*eux.dag()+euy*euy.dag())
HOrb = tensor(HOrb, qeye(N_spins))

In [5]:
#Spin-Orbit coupling
SO_g = 46
SO_e = 250
HSO = -SO_g/2*tensor(sgy, su*su.dag() - sd*sd.dag()) 
HSO+= -SO_e/2*tensor(suy, su*su.dag() - sd*sd.dag()) 

In [6]:
#Static strain coupling
delta_g = 0
alpha_g = 0
beta_g = 0
HStr_g = (delta_g+alpha_g)*egx*egx.dag()
HStr_g += beta_g*egx*egy.dag()
HStr_g += beta_g*egy*egx.dag()
HStr_g += (delta_g-alpha_g)*egy*egy.dag()

delta_e = 0
alpha_e = 0
beta_e = 0
HStr_e = (delta_e+alpha_e)*eux*eux.dag()
HStr_e += beta_e*eux*euy.dag()
HStr_e += beta_e*euy*eux.dag()
HStr_e += (delta_e-alpha_e)*euy*euy.dag()

HStr = HStr_g + HStr_e
HStr = tensor(HStr, qeye(N_spins))

In [7]:
T1opt = 1.74
T1orb_e = 200e-3
T1orb_g = 40
T1spin = 1000

egp = egx+1j*egy
egp = egp/egp.norm()

egm = -1j*(egx-1j*egy)
egm = egm/egm.norm()

eup = eux+1j*euy
eup = eup/eup.norm()

eum = -1j*(eux-1j*euy)
eum = eum/eum.norm()

c_ops = []

# Neglect optical excitation from the environment --> only photon emission stimulated by photon bath @5K
# Assume all optical relaxations to be equally fast
c_ops.append(np.sqrt(1/(2*np.pi*T1opt))*tensor(egp*eup.dag(), qeye(N_spins)))
c_ops.append(np.sqrt(1/(2*np.pi*T1opt))*tensor(egm*eum.dag(), qeye(N_spins)))
c_ops.append(np.sqrt(1/4*1/(2*np.pi*T1opt))*tensor((1-1j)*egp*eum.dag(), qeye(N_spins)))
c_ops.append(np.sqrt(1/4*1/(2*np.pi*T1opt))*tensor((1+1j)*egm*eup.dag(), qeye(N_spins)))

# Spin decay
c_ops.append( tensor( qeye(N_orbs), np.sqrt(1/(2*np.pi*T1spin))*sd*su.dag() ) )
c_ops.append( tensor( qeye(N_orbs), np.sqrt(1/(2*np.pi*T1spin)*boltzmann(0, T))*su*sd.dag() ) )

# Orbital decay in the groundstate manifold
c_ops.append(np.sqrt(1/(T1orb_g))*tensor(egm*egp.dag(), sd*sd.dag())) 
c_ops.append(np.sqrt(1/(T1orb_g)*boltzmann(SO_g, T))*tensor(egp*egm.dag(), sd*sd.dag())) 
c_ops.append(np.sqrt(1/(T1orb_g))*tensor(egp*egm.dag(), su*su.dag())) 
c_ops.append(np.sqrt(1/(T1orb_g)*boltzmann(SO_g, T))*tensor(egm*egp.dag(), su*su.dag())) 

# Orbital decay in the excited manifold
c_ops.append(np.sqrt(1/(T1orb_e))*tensor(eum*eup.dag(), sd*sd.dag())) 
c_ops.append(np.sqrt(1/(T1orb_e)*boltzmann(SO_e, T))*tensor(eup*eum.dag(), sd*sd.dag())) 
c_ops.append(np.sqrt(1/(T1orb_e))*tensor(eup*eum.dag(), su*su.dag())) 
c_ops.append(np.sqrt(1/(T1orb_e)*boltzmann(SO_e, T))*tensor(eum*eup.dag(), su*su.dag())) 

In [9]:
nus = np.arange(nu_Orb-2*(SO_e+SO_g), nu_Orb+2*(SO_e+SO_g), 0.1)
spectr_ops = [
    tensor(egp*eup.dag(), su*su.dag()),
    tensor(egp*eup.dag(), sd*sd.dag()),
    tensor(egm*eup.dag(), su*su.dag()),
    tensor(egm*eup.dag(), sd*sd.dag()),
    tensor(egp*eum.dag(), su*su.dag()),
    tensor(egp*eum.dag(), sd*sd.dag()),
    tensor(egm*eum.dag(), su*su.dag()),
    tensor(egm*eum.dag(), sd*sd.dag())
]

q = 0
Bs = np.linspace(1,10,5)
spectr = np.zeros((len(Bs), len(nus), len(spectr_ops)))
for i, B in enumerate(Bs):
    # Note that Lz is defined with -sigma_y in the original basis of ex, ey meaning Lz = i|ex><ey| - i|ey><ex| = -sy
    HSys = HOrb + HSO -q*mu_B/(2*h)*B*tensor(-sgy-suy, qeye(N_spins)) - mu_B*B/h*tensor(qeye(N_orbs), Sz)
    for j, spectr_op in enumerate(spectr_ops):
        spectr_ = np.abs(spectrum(HSys, -nus, c_ops, spectr_op, spectr_op.trans()))
        spectr[i,:,j] += spectr_/max(spectr_)


AxisError: axis 2 is out of bounds for array of dimension 2

In [ ]:
plt.plot(nus, np.sum(spectr[0, :, :], axis=2))

In [149]:
Hint = 1 * (2*egp*eup.dag() + 2*egm*eum.dag() + egp*eum.dag()*(1-1j) + egm*eup.dag()*(1+1j))
Hint += Hint.trans()
Hint = tensor(Hint, qeye(N_spins))

energies, eigsts = HSys.eigenstates()
Heig = Qobj()
for energy, eigenstate in zip(energies, eigenstates):
    Heig += energy*eigenstate*eigenstate.dag()
Heig-Hint

commutator(Hint, Hint)

Quantum object: dims = [[4, 2], [4, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]